<a href="https://colab.research.google.com/github/fasthill/ML-DL-study-alone/blob/main/5-1%20%EA%B2%B0%EC%A0%95%20%ED%8A%B8%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Analyse from XGBoost

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-1.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce

In [2]:
from xgboost import XGBClassifier
import xgboost

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [12]:
from sklearn.linear_model import LogisticRegression, SGDClassifier, SGDRegressor
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [4]:
# confusion matrix to list 변환
def matrix_to_list(confu_matrix):
    m_list = []
    tn = confu_matrix[0,0]
    fp = confu_matrix[0,1]
    fn = confu_matrix[1,0]
    tp = confu_matrix[1,1]
    m_list.extend([tn, fp, fn, tp])
    return m_list

In [5]:
def predict_p(test_target, y_predict_list): 
    ps = precision_score(test_target, y_predict_list)
    rs = recall_score(test_target, y_predict_list)
    fs = f1_score(test_target, y_predict_list)
    roc = roc_auc_score(test_target, y_predict_list)
#     cm = matrix_to_list(confusion_matrix(test_target, y_predict_list))
    collect_list = [ps, rs, fs, roc]
#     collect_list.extend(cm)
    return collect_list

In [10]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds, show_progress=False)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['Disbursed'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
#     #Print model report:
#     print("\nModel Report")
#     print("Accuracy : %.4g" % metrics.accuracy_score(dtrain['Disbursed'].values, dtrain_predictions))
#     print("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Disbursed'], dtrain_predprob))
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [ ]:
# #Choose all predictors except target & IDcols
# predictors = [x for x in train.columns if x not in [target, IDcol]]
# xgb1 = XGBClassifier(
#  learning_rate =0.1,
#  n_estimators=1000,
#  max_depth=5,
#  min_child_weight=1,
#  gamma=0,
#  subsample=0.8,
#  colsample_bytree=0.8,
#  objective= 'binary:logistic',
#  nthread=4,
#  scale_pos_weight=1,
#  seed=27)
# modelfit(xgb1, train, predictors)

In [ ]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(train[predictors],train[target])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [6]:
code = {'005930' : ['삼성전자', 'sec'], '373220' : ['LG에너지솔루션', 'lgenergy'], 
        '000660' : ['SK하이닉스', 'skhinix'], '207940' : ['삼성바이오로직스', 'ssbio'],
        '006400' : ['삼성SDI', 'sdi'], '051910' : ['LG화학', 'lgchemical'],
        '005935' : ['삼성전자우', 'secpre'], '005380' : ['현대차', 'hyunmotor'],
        '035420' : ['NAVER', 'naver'], '000270' : ['기아','kia'],
        '035720' : ['카카오', 'kakao'], '005490' : ['POSCO홀딩스', 'poscoholding'],
        '105560' : ['KB금융', 'kbbank'], '028260' : ['삼성물산', 'sscnt'],
        '068270' : ['셀트리온', 'celltrion'], '012330' : ['현대모비스', 'mobis'],
        '055550' : ['신한지주', 'shgroup'], '066570' : ['LG전자', 'lgelec'],
        '003670' : ['포스코케미칼', 'poscochemical'], '096770' : ['SK이노베이션', 'skinnovation'],
        '033780' : ['KT&G', 'ktng']}

code = {'005930' : ['삼성전자', 'sec']}

In [7]:
fname = f'df_sec_sel.pkl'
stock_name = 'sec'
directory_for_ml = '../data/data_for_ml/'
f_name = directory_for_ml + fname
df = pd.read_pickle(f_name) 

# train, val,: 8, test: 2
split_ratio = 0.8
split_n = int(len(df)*split_ratio)

data = df.iloc[:split_n, :-5]
target = df.iloc[:split_n, -4]
test_input = df.iloc[split_n:, :-5]
test_target = df.iloc[split_n:, -4]

train_input, val_input, train_target, val_target = \
     train_test_split(data, target, random_state=42, test_size=0.2, stratify=target)

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
val_scaled = ss.transform(val_input)
test_scaled = ss.transform(test_input)

In [50]:
xgb = None
# xgb = xgboost.XGBRegressor(random_state=42, n_estimator=100, learning_rate=0.01)
xgb = xgboost.XGBClassifier(random_state=42, use_label_encoder=False,
                            learning_rate =0.22, n_estimators=500, 
#                             max_depth=5, 
                            min_child_weight=6, # imbalance 경감
                            gamma=0, subsample=0.9, 
                            colsample_bytree=0.8,
                            objective= 'binary:logistic', 
#                             nthread=4, # Gpu 수
                            max_delta_step = 0.5, # best value found
                            scale_pos_weight=40,) # class imbalance 경감
xgb.fit(train_scaled, train_target, eval_metric = 'logloss')
y_pred = xgb.predict(val_scaled)
# val_error = mean_squared_error(val_target, y_pred) # 책에 없음
# print("Validation MSE:", val_error)           # 책에 없음

train_score_xgb = xgb.score(train_scaled, train_target)
val_score_xgb = xgb.score(val_scaled, val_target)
test_score_xgb = xgb.score(test_scaled, test_target)
ps, rs, fs, roc = predict_p(test_target, xgb.predict(test_scaled))
cm = confusion_matrix(test_target, xgb.predict(test_scaled))

print("train accuracy: {:.4f}, val accuracy {:.4f}, test accuracy {:.4f}".
      format(train_score_xgb, val_score_xgb, test_score_xgb))
print("precision : {:.4f}, recall : {:.4f}, f1score : {:.4f}, roc : {:.4f}".
     format (ps, rs, fs, roc))
print(cm)

train accuracy: 0.9826, val accuracy 0.8140, test accuracy 0.8519
precision : 0.7143, recall : 0.8824, f1score : 0.7895, roc : 0.8601
[[31  6]
 [ 2 15]]


In [55]:
param_test1 = {
    'max_depth':range(3,10,2),
    'min_child_weight':range(3,8,2),
    'subsample':[0.1, 0.3, 0.5, 0.7, 0.9],
    'subsample':[0.1, 0.3, 0.5, 0.7, 0.9],
    'scale_pos_weight': range(20, 50, 5),
    'max_delta_step': [0.1, 0.3, 0.5, 0.7, 0.9],
    'learning_rate': [0.1, 0.3, 0.5, 0.7, 0.9, 1.1, 1.3, 1.5, 1.7, 1.9]
}
gsearch1 = GridSearchCV(
    estimator = XGBClassifier(random_state=42, use_label_encoder=False,
                            learning_rate =0.22, n_estimators=500, 
#                             max_depth=5, 
                            min_child_weight=6, # imbalance 경감
                            gamma=0, subsample=0.9, 
                            colsample_bytree=0.8,
                            objective= 'binary:logistic', 
#                             nthread=4, # Gpu 수
                            max_delta_step = 0.5, # best value found
                            scale_pos_weight=40,), # class imbalance 경감, 
    param_grid = param_test1, scoring='roc_auc', cv=5)
#     param_grid = param_test1, scoring='roc_auc',n_jobs=4, iid=False, cv=5),
    
gsearch1.fit(train_scaled, train_target, eval_metric = 'logloss')
gsearch1.best_params_, gsearch1.best_score_

TypeError: 'float' object cannot be interpreted as an integer

In [ ]:
gsearch1.

In [53]:
for i in ['mean_test_score', 'std_test_score']:
        print(i," : ", gsearch1.cv_results_[i])

mean_test_score  :  [0.5        0.91806667        nan 0.5        0.90966667        nan
 0.5        0.91546667        nan 0.5        0.91293333        nan
 0.5        0.90963333        nan 0.5        0.9145            nan
 0.5        0.90633333        nan 0.5        0.9128            nan
 0.5        0.9112            nan 0.5        0.91456667        nan
 0.5        0.9088            nan 0.5        0.91283333        nan
 0.5        0.90803333        nan 0.5        0.91466667        nan
 0.5        0.90866667        nan 0.5        0.90796667        nan
 0.5        0.91706667        nan 0.5        0.92043333        nan
 0.5        0.90883333        nan 0.5        0.90966667        nan
 0.5        0.91113333        nan 0.5        0.89726667        nan
 0.5        0.89386667        nan 0.5        0.9096            nan
 0.5        0.90566667        nan 0.5        0.90386667        nan
 0.5        0.90383333        nan 0.5        0.91293333        nan
 0.5        0.90873333        nan 0.5     

In [36]:
gsearch1.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'param_min_child_weight', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [52]:
# use best fit model using .best_estimator

best_xgb = gsearch1.best_estimator_
y_pred = best_xgb.predict(val_scaled)
# val_error = mean_squared_error(val_target, y_pred) # 책에 없음
# print("Validation MSE:", val_error)           # 책에 없음

train_score_xgb = best_xgb.score(train_scaled, train_target)
val_score_xgb = best_xgb.score(val_scaled, val_target)
test_score_xgb = best_xgb.score(test_scaled, test_target)
ps, rs, fs, roc = predict_p(test_target, best_xgb.predict(test_scaled))
cm = confusion_matrix(test_target, best_xgb.predict(test_scaled))

print("train accuracy: {:.4f}, val accuracy {:.4f}, test accuracy {:.4f}".
      format(train_score_xgb, val_score_xgb, test_score_xgb))
print("precision : {:.4f}, recall : {:.4f}, f1score : {:.4f}, roc : {:.4f}".
     format (ps, rs, fs, roc))
print(cm)

train accuracy: 0.9884, val accuracy 0.8140, test accuracy 0.8148
precision : 0.6667, recall : 0.8235, f1score : 0.7368, roc : 0.8172
[[30  7]
 [ 3 14]]
